## Content

1. [Methods](#Methods)
2. [Problem to QUBO](#Problem-to-QUBO)
3. [Gate QUBO](#Gate-QUBO)
4. [Exact solver](#Exact-solver)
5. [Results 3 qubits](#Results-3-qubits)
6. [Results 6 qubits](#Results-6-qubits)
7. [Results 13 qubits](#Results-13-qubits)
8. [Results 140 qubits on a Hybrid Solver](#Results-140-qubits-on-a-Hybrid-Solver)

### Methods

In [1]:
import numpy as np
from pyqubo import Binary as bit
import dimod
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite
import dwave.inspector
from dwave.system import LeapHybridSampler

### Problem to QUBO

In [4]:
p1, p2, p3 = 39.5, 3.6, 87.8
N = 100

In [5]:
q1, q2, q3 = bit("q1"), bit("q2"), bit("q3")

In [6]:
function = (- N + p1 * q1 + p2 * q2 +  p3 * q3)**2
model = function.compile()

In [7]:
L = model.to_qubo()
L

({('q1', 'q1'): -6339.75,
  ('q3', 'q1'): 6936.2,
  ('q2', 'q1'): 284.40000000000003,
  ('q3', 'q3'): -9851.16,
  ('q3', 'q2'): 632.16,
  ('q2', 'q2'): -707.04},
 10000.0)

In [8]:
a1, a2, a3 = L[0][('q1', 'q1')], L[0][('q2', 'q2')], L[0][('q3', 'q3')] 
b12, b13, b23 =  L[0][('q2', 'q1')], L[0][('q3', 'q1')], L[0][('q3', 'q2')]

In [9]:
b23

632.16

### Exact solver

We can calculate all the $2^3$ possible solutions using the _ExactSolver_ method. Note that this method (solver) should be used for problems with a small number of variables.

In [10]:
ex_sampler = dimod.ExactSolver()
qubo_sample_set = ex_sampler.sample_qubo(L[0])

In [11]:
print(qubo_sample_set)

  q1 q2 q3   energy num_oc.
4  0  1  1 -9926.04       1
3  0  0  1 -9851.16       1
2  1  0  1 -9254.71       1
5  1  1  1 -9045.19       1
6  1  1  0 -6762.39       1
1  1  0  0 -6339.75       1
7  0  1  0  -707.04       1
0  0  0  0      0.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


The solution is to allocate on asset $q_2$ and $q_3\,.$ Alternative, we can set as a Binary Quadratic Model (BQM).

In [12]:
bqm = model.to_bqm()
bqm_sample_set = ex_sampler.sample(bqm)
print(bqm_sample_set)

  q1 q2 q3  energy num_oc.
6  0  1  1   73.96       1
1  0  0  1  148.84       1
2  1  0  1  745.29       1
5  1  1  1  954.81       1
4  1  1  0 3237.61       1
3  1  0  0 3660.25       1
7  0  1  0 9292.96       1
0  0  0  0 10000.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


The optimal solution $q_1 = 0,\, q_2 = 1,\, q_3 = 1$ implies that $100 - 91.3 = 8.7$ EUR is not allocated.


### Results 3 qubits

Let's see the proprieties of a quantum annealer process with a _Pegasus_ topology. See __[Pegasus Graph](https://docs.dwavesys.com/docs/latest/c_gs_4.html#topology-intro-pegasus)__ for more information.

In [13]:
qpu_sampler = DWaveSampler(solver={'topology__type': 'pegasus'})

In [14]:
list(qpu_sampler.properties)

['num_qubits',
 'qubits',
 'couplers',
 'h_range',
 'j_range',
 'supported_problem_types',
 'parameters',
 'vfyc',
 'anneal_offset_ranges',
 'anneal_offset_step',
 'anneal_offset_step_phi0',
 'annealing_time_range',
 'chip_id',
 'default_annealing_time',
 'default_programming_thermalization',
 'default_readout_thermalization',
 'extended_j_range',
 'h_gain_schedule_range',
 'max_anneal_schedule_points',
 'max_h_gain_schedule_points',
 'num_reads_range',
 'per_qubit_coupling_range',
 'problem_run_duration_range',
 'problem_timing_data',
 'programming_thermalization_range',
 'readout_thermalization_range',
 'tags',
 'topology',
 'category',
 'quota_conversion_rate']

In [15]:
qpu_sampler.properties['chip_id']

'Advantage_system6.3'

In [16]:
qpu_sampler.properties['num_qubits']

5760

This QPU (_Advantage_system6.2_) has $5760$ qubits, which is a significant improvement related to D-Waves 2000Q processor (_Chimera_ graph and $2048$ qubits). Note that the problem bias (linear) and coupling (quadratic) coefficients are rescaled to a range of allowed values. The problem is first converted to a Ising model then all coefficient are divided by

$$\max \left\{\max \left( \frac{\max (h)}{\max (h\_range)}, 0 \right), \max \left( \frac{\min (h)}{\min(h\_range)}, 0 \right), \max \left( \frac{\max(J)}{\max(J\_range)}, 0 \right), \max \left( \frac{\min(J)}{\min(J\_range)}, 0 \right) \right\}.$$

See __[auto_scale](https://docs.dwavesys.com/docs/latest/c_solver_parameters.html#param-autoscale)__ D-Wave's documentation.

In [17]:
qpu_sampler.properties['h_range']

[-4.0, 4.0]

In [18]:
qpu_sampler.properties['j_range']

[-1.0, 1.0]

The _default_annealing_time_ (microseconds $\mu s$) for one read is of particular interest given the $1$ minute = $60,000,000$ $\mu s$ available to run problems with a trial plan account type. 

In [19]:
qpu_sampler.properties["default_annealing_time"]    

20.0

See  __[Solver Properties](https://docs.dwavesys.com/docs/latest/c_solver_properties.html#)__ for more information. We can also see the possible input parameters when solving a problem.

In [20]:
list(qpu_sampler.parameters.keys())

['anneal_offsets',
 'anneal_schedule',
 'annealing_time',
 'answer_mode',
 'auto_scale',
 'flux_biases',
 'flux_drift_compensation',
 'h_gain_schedule',
 'initial_state',
 'max_answers',
 'num_reads',
 'programming_thermalization',
 'readout_thermalization',
 'reduce_intersample_correlation',
 'reinitialize_state',
 'warnings',
 'label']

Run the problem without the need to embed the problem into the graph topology.

In [21]:
pegasus_sampler = EmbeddingComposite(qpu_sampler)

We will run $3$ times with a $3 \mu s$ per read. Note that the default is $20 \mu s\,.$

In [22]:
sample_results = pegasus_sampler.sample(bqm, num_reads=3, annealing_time=3, label='qubo_3')

In [23]:
print(sample_results)

  q1 q2 q3 energy num_oc. chain_.
0  0  1  1  73.96       2     0.0
1  0  0  1 148.84       1     0.0
['BINARY', 2 rows, 3 samples, 3 variables]


$2$ out of $3$ samples (reads) resulted in the optimal solution $q_1=0, q_2=1, q_3=1\,.$ The _chain_ value is the percentage of chain breaks. In this toy example it is expected to be zero, since the _Pegasus_ topology can embed a 3-loop (triangle graph) without additional physical qubits per logical qubit.

We can confirm the quantum annealing time and other information.

In [24]:
sample_results.info

{'timing': {'qpu_sampling_time': 208.8,
  'qpu_anneal_time_per_sample': 3.0,
  'qpu_readout_time_per_sample': 46.06,
  'qpu_access_time': 16137.97,
  'qpu_access_overhead_time': 838.03,
  'qpu_programming_time': 15929.17,
  'qpu_delay_time_per_sample': 20.54,
  'total_post_processing_time': 1679.0,
  'post_processing_overhead_time': 1679.0},
 'problem_id': 'e144dd9c-56c1-42fd-90d7-80a29d72a370',
 'problem_label': 'qubo_3',
 'embedding_context': {'embedding': {'q1': (5508,),
   'q3': (5493,),
   'q2': (584,)},
  'chain_break_method': 'majority_vote',
  'embedding_parameters': {},
  'chain_strength': 2011.9788242097256},
 'warnings': []}

Use _dwave.inspector_  to visualize which qubits on the traged QPU where used to solve this problem. Note that _dwave.inspector.show_ is an interactive window connected to the server.

In [25]:
dwave.inspector.show(sample_results)

Serving Inspector on http://127.0.0.1:18000/?problemId=e144dd9c-56c1-42fd-90d7-80a29d72a370

'http://127.0.0.1:18000/?problemId=e144dd9c-56c1-42fd-90d7-80a29d72a370'

### Results 6 qubits

In [26]:
add_qubits = 3

np.random.seed(0)

qubits = [bit("q1"), bit("q2"), bit("q3")]

p1, p2, p3 = 39.5, 3.6, 87.8
N = 100

asset_prices = [p1, p2, p3]
function = -N + p1 * qubits[0] + p2 * qubits[1] +  p3 * qubits[2]

for i in range(add_qubits):
    
    delta = np.random.uniform(size=1)
    price = 110 * (1 + delta[0]) 
    asset_prices.append(price)
    
    qubits.append(bit(f"q{4 + i}"))
    
    function += price * bit(f"q{4 + i}")
    
square_function = function**2

In [27]:
asset_prices

[39.5, 3.6, 87.8, 170.36948543200572, 188.67083030096614, 176.30397136788085]

In [28]:
function

((176.303971 * Binary('q6')) + (188.670830 * Binary('q5')) + (170.369485 * Binary('q4')) + (87.800000 * Binary('q3')) + (3.600000 * Binary('q2')) + -100.000000 + (39.500000 * Binary('q1')))

In [29]:
model_6_qb = square_function.compile()
bqm_6_qb = model_6_qb.to_bqm()
bqm_6_qb

BinaryQuadraticModel({'q5': -2137.4838537372634, 'q1': -6339.75, 'q6': -4177.703953489618, 'q4': -5048.135520034735, 'q2': -707.04, 'q3': -9851.16}, {('q1', 'q5'): 14904.995593776326, ('q6', 'q5'): 66526.83332667168, ('q6', 'q1'): 13928.013738062587, ('q4', 'q5'): 64287.504548809746, ('q4', 'q1'): 13459.189349128452, ('q4', 'q6'): 60073.63376312986, ('q2', 'q5'): 1358.4299781669563, ('q2', 'q1'): 284.40000000000003, ('q2', 'q6'): 1269.3885938487422, ('q2', 'q4'): 1226.6602951104412, ('q3', 'q5'): 33130.59780084965, ('q3', 'q1'): 6936.2, ('q3', 'q6'): 30958.977372199875, ('q3', 'q4'): 29916.881641860204, ('q3', 'q2'): 632.16}, 10000.0, 'BINARY')

In [30]:
nr = 10
at = 20
cs = 100000
name = f"qubo_{nr}_t{at}"
sample_results_6_qb = pegasus_sampler.sample(bqm_6_qb, num_reads=nr, annealing_time=at, chain_strength =cs,label=name)

In [31]:
print(sample_results_6_qb)

  q1 q2 q3 q4 q5 q6       energy num_oc. chain_.
0  0  1  1  0  0  0        73.96       1     0.0
1  0  0  1  0  0  0       148.84       2     0.0
2  1  0  1  0  0  0       745.29       1     0.0
3  1  1  1  0  0  0       954.81       2     0.0
4  0  0  0  1  0  0   4951.86448       1     0.0
5  0  1  0  1  0  0  5471.484775       1     0.0
6  1  0  0  1  0  0 12071.303829       1     0.0
7  0  1  1  0  0  1 28124.622013       1     0.0
['BINARY', 8 rows, 10 samples, 6 variables]


In [32]:
sample_results_6_qb

SampleSet(rec.array([([0, 1, 1, 0, 0, 0],    73.96      , 1, 0.),
           ([0, 0, 1, 0, 0, 0],   148.84      , 2, 0.),
           ([1, 0, 1, 0, 0, 0],   745.29      , 1, 0.),
           ([1, 1, 1, 0, 0, 0],   954.81      , 2, 0.),
           ([0, 0, 0, 1, 0, 0],  4951.86447997, 1, 0.),
           ([0, 1, 0, 1, 0, 0],  5471.48477508, 1, 0.),
           ([1, 0, 0, 1, 0, 0], 12071.30382909, 1, 0.),
           ([0, 1, 1, 0, 0, 1], 28124.62201256, 1, 0.)],
          dtype=[('sample', 'i1', (6,)), ('energy', '<f8'), ('num_occurrences', '<i4'), ('chain_break_fraction', '<f8')]), Variables(['q1', 'q2', 'q3', 'q4', 'q5', 'q6']), {'timing': {'qpu_sampling_time': 958.6, 'qpu_anneal_time_per_sample': 20.0, 'qpu_readout_time_per_sample': 55.32, 'qpu_access_time': 16887.77, 'qpu_access_overhead_time': 341.23, 'qpu_programming_time': 15929.17, 'qpu_delay_time_per_sample': 20.54, 'total_post_processing_time': 1692.0, 'post_processing_overhead_time': 1692.0}, 'problem_id': '8c0a7da0-f001-4ddd-8800-a

In [33]:
sample_results_6_qb.info

{'timing': {'qpu_sampling_time': 958.6,
  'qpu_anneal_time_per_sample': 20.0,
  'qpu_readout_time_per_sample': 55.32,
  'qpu_access_time': 16887.77,
  'qpu_access_overhead_time': 341.23,
  'qpu_programming_time': 15929.17,
  'qpu_delay_time_per_sample': 20.54,
  'total_post_processing_time': 1692.0,
  'post_processing_overhead_time': 1692.0},
 'problem_id': '8c0a7da0-f001-4ddd-8800-ab6a6dfa68d3',
 'problem_label': 'qubo_10_t20',
 'embedding_context': {'embedding': {'q1': (2749,),
   'q5': (2689, 3764),
   'q6': (3749,),
   'q4': (2734,),
   'q2': (2674, 3809),
   'q3': (3734,)},
  'chain_break_method': 'majority_vote',
  'embedding_parameters': {},
  'chain_strength': 100000},
 'warnings': []}

In [34]:
dwave.inspector.show(sample_results_6_qb)

Serving Inspector on http://127.0.0.1:18000/?problemId=8c0a7da0-f001-4ddd-8800-ab6a6dfa68d3

'http://127.0.0.1:18000/?problemId=8c0a7da0-f001-4ddd-8800-ab6a6dfa68d3'

### Results 13 qubits

In [35]:
add_qubits = 10

np.random.seed(1)

qubits = [bit("q1"), bit("q2"), bit("q3")]

p1, p2, p3 = 39.5, 3.6, 87.8
N = 100

asset_prices = [p1, p2, p3]
function = -N + p1 * qubits[0] + p2 * qubits[1] +  p3 * qubits[2]

for i in range(add_qubits):
    
    delta = np.random.uniform(size=1)
    price = 110 * (1 + delta[0]) 
    asset_prices.append(price)
    
    qubits.append(bit(f"q{4 + i}"))
    
    function += price * bit(f"q{4 + i}")
    
square_function = function**2

In [36]:
asset_prices

[39.5,
 3.6,
 87.8,
 155.87242051728316,
 189.23569427863737,
 110.01258122990794,
 143.25658298950236,
 126.14314798988245,
 120.15724542456776,
 130.4886232515438,
 148.01167997473527,
 153.6444221653737,
 169.26984074036926]

In [37]:
function

((169.269841 * Binary('q13')) + (153.644422 * Binary('q12')) + (148.011680 * Binary('q11')) + (130.488623 * Binary('q10')) + (120.157245 * Binary('q9')) + (126.143148 * Binary('q8')) + (143.256583 * Binary('q7')) + (110.012581 * Binary('q6')) + (189.235694 * Binary('q5')) + (155.872421 * Binary('q4')) + (87.800000 * Binary('q3')) + (3.600000 * Binary('q2')) + -100.000000 + (39.500000 * Binary('q1')))

In [38]:
model_13_qb = square_function.compile()
bqm_13_qb = model_13_qb.to_bqm()
bqm_13_qb

BinaryQuadraticModel({'q7': -8128.868028072295, 'q5': -2036.9908666095725, 'q12': -7122.27597054316, 'q6': -9899.748217114495, 'q1': -6339.75, 'q4': -6878.272625539874, 'q2': -707.04, 'q3': -9851.16, 'q13': -5201.689163803876, 'q8': -9316.535813179105, 'q11': -7694.878586003604, 'q10': -9070.44385222542, 'q9': -9593.68545689374}, {('q5', 'q7'): 54218.51788400742, ('q12', 'q7'): 44021.149829615984, ('q12', 'q5'): 58150.017801009104, ('q6', 'q7'): 31520.05294570335, ('q6', 'q5'): 41636.614376853235, ('q6', 'q12'): 33805.638947980886, ('q1', 'q7'): 11317.270056170686, ('q1', 'q5'): 14949.619848012351, ('q1', 'q12'): 12137.909351064522, ('q1', 'q6'): 8690.993917162727, ('q4', 'q7'): 44659.500691217574, ('q4', 'q5'): 58993.2514309596, ('q4', 'q12'): 47897.85596379222, ('q4', 'q6'): 34295.85464731997, ('q4', 'q1'): 12313.92122086537, ('q2', 'q7'): 1031.447397524417, ('q2', 'q5'): 1362.4969988061891, ('q2', 'q12'): 1106.2398395906907, ('q2', 'q6'): 792.0905848553372, ('q2', 'q1'): 284.4000000

In [39]:
nr = 20
at = 20
cs = 100000
name = f"qubo_{nr}_t{at}"
sample_results_13_qb = pegasus_sampler.sample(bqm_13_qb, num_reads=nr, annealing_time=at, chain_strength =cs,label=name)

In [40]:
print(sample_results_13_qb)

   q1 q10 q11 q12 q13 q2 q3 q4 q5 q6 q7 q8 q9        energy num_oc. chain_.
0   0   0   0   0   0  1  0  0  0  1  0  0  0    185.302368       1     0.0
1   0   0   0   0   0  0  0  0  0  0  0  1  0    683.464187       2     0.0
2   0   0   0   0   0  1  0  0  0  0  0  1  0    884.654852       1     0.0
3   1   0   0   0   0  0  0  0  0  0  0  0  1   3558.986932       1     0.0
4   1   0   1   0   0  1  0  0  0  0  0  0  0   8301.338228       1     0.0
5   0   0   0   0   0  0  1  0  0  1  0  0  0   9567.301047       1     0.0
6   0   0   0   0   0  1  1  0  0  1  0  0  0  10284.511632       1     0.0
7   0   0   0   0   0  1  1  0  0  0  0  0  1  12445.019007       1     0.0
8   0   0   0   0   0  0  1  0  0  0  0  1  0  12983.040974       1     0.0
9   0   0   0   0   0  1  1  0  0  0  0  1  0  13816.391639       1     0.0
10  0   1   0   0   0  0  1  0  0  0  0  0  0  13992.198391       1     0.0
11  0   0   0   0   0  0  0  0  0  1  0  1  0  18538.382599       2     0.0
12  0   1   

In [41]:
sample_results_13_qb

SampleSet(rec.array([([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],    185.30236774, 1, 0.),
           ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],    683.46418682, 2, 0.),
           ([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],    884.65485235, 1, 0.),
           ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],   3558.98693165, 1, 0.),
           ([1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],   8301.33822782, 1, 0.),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],   9567.30104686, 1, 0.),
           ([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0],  10284.51163171, 1, 0.),
           ([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],  12445.01900672, 1, 0.),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],  12983.04097384, 1, 0.),
           ([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0],  13816.39163937, 1, 0.),
           ([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],  13992.19839075, 1, 0.),
           ([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],  18538.38259937, 2, 0.),
           ([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],  

In [60]:
round(3.5)

4

In [42]:
sample_results_13_qb.info

{'timing': {'qpu_sampling_time': 1957.2,
  'qpu_anneal_time_per_sample': 20.0,
  'qpu_readout_time_per_sample': 57.32,
  'qpu_access_time': 17885.57,
  'qpu_access_overhead_time': 827.43,
  'qpu_programming_time': 15928.37,
  'qpu_delay_time_per_sample': 20.54,
  'total_post_processing_time': 96.0,
  'post_processing_overhead_time': 96.0},
 'problem_id': '544d84e6-d4f3-43b2-9ad9-3b3e56daac31',
 'problem_label': 'qubo_20_t20',
 'embedding_context': {'embedding': {'q5': (5236, 297),
   'q7': (5161, 372, 373),
   'q12': (327, 5146),
   'q6': (5296, 5297),
   'q1': (357,),
   'q4': (5282, 5281),
   'q2': (282, 5251),
   'q3': (312, 5131),
   'q13': (447, 5266),
   'q8': (387, 5176, 388),
   'q11': (342,),
   'q10': (5221, 432),
   'q9': (267, 5205, 5206)},
  'chain_break_method': 'majority_vote',
  'embedding_parameters': {},
  'chain_strength': 100000},
 'warnings': []}

In [43]:
dwave.inspector.show(sample_results_13_qb)

Serving Inspector on http://127.0.0.1:18000/?problemId=544d84e6-d4f3-43b2-9ad9-3b3e56daac31

'http://127.0.0.1:18000/?problemId=544d84e6-d4f3-43b2-9ad9-3b3e56daac31'

### Results 140 qubits on a Hybrid Solver

In [44]:
add_qubits = 137

np.random.seed(1)

qubits = [bit("q1"), bit("q2"), bit("q3")]

p1, p2, p3 = 39.5, 3.6, 87.8
N = 100

asset_prices = [p1, p2, p3]
function = -N + p1 * qubits[0] + p2 * qubits[1] +  p3 * qubits[2]

for i in range(add_qubits):
    
    delta = np.random.uniform(size=1)
    price = 110 * (1 + delta[0]) 
    asset_prices.append(price)
    
    qubits.append(bit(f"q{4 + i}"))
    
    function += price * bit(f"q{4 + i}")
    
square_function = function**2

In [58]:
print(asset_prices)

[39.5, 3.6, 87.8, 155.87242051728316, 189.23569427863737, 110.01258122990794, 143.25658298950236, 126.14314798988245, 120.15724542456776, 130.4886232515438, 148.01167997473527, 153.6444221653737, 169.26984074036926, 156.1113965843624, 185.37414504364355, 132.48974747046694, 206.592918003004, 113.01263525177188, 183.75142611962423, 155.90352826038398, 171.4558811290327, 125.44256324547572, 131.79116379933666, 198.08190255430904, 216.50877332913373, 144.47665959751671, 186.15548772362456, 206.4028067525642, 208.40673298542322, 119.35486325067558, 114.29602615561704, 128.6813461521026, 206.59567537723547, 120.8181517216355, 156.32183875055574, 215.3678483165552, 168.64818134703188, 186.10648253455207, 144.70671941066692, 185.5151020449742, 201.80882390871102, 112.01171050786108, 192.51587464394643, 218.77471977971442, 192.29822198178232, 140.84883912708457, 196.82072612966374, 121.35486072354063, 159.26828787934957, 209.94550534024052, 142.29755632110474, 141.65528724449834, 124.303142933

In [56]:
model_102_qb = square_function.compile()
bqm_102_qb = model_102_qb.to_bqm()

In [48]:
hb_sampler = LeapHybridSampler(solver={'category': 'hybrid'})

In [49]:
hb_sampler.properties

{'minimum_time_limit': [[1, 3.0],
  [1024, 3.0],
  [4096, 10.0],
  [10000, 40.0],
  [30000, 200.0],
  [100000, 600.0],
  [1000000, 600.0]],
 'maximum_time_limit_hrs': 24.0,
 'maximum_number_of_variables': 1000000,
 'maximum_number_of_biases': 200000000,
 'parameters': {'time_limit': 'Maximum requested runtime in seconds.'},
 'supported_problem_types': ['bqm'],
 'category': 'hybrid',
 'version': '2.2',
 'quota_conversion_rate': 20}

In [50]:
hb_sampler.parameters

{'time_limit': ['parameters'], 'label': []}

In [51]:
tl = 20
hb_name = f'hb_qubo_{len(asset_prices)}_t{tl}'
hb_sample_results = hb_sampler.sample(bqm_102_qb, time_limit=tl, label=hb_name)

In [52]:
print(hb_sample_results)

  q1 q10 q100 q101 q102 q103 q104 q105 q106 q107 q108 ... q99 energy num_oc.
0  0   0    0    0    0    0    0    0    0    0    0 ...   0  73.96       1
['BINARY', 1 rows, 1 samples, 140 variables]


In [53]:
qubits = [i for i in hb_sample_results.first]
solution = [ (k, qubits[0][k]) for k in qubits[0] if qubits[0][k] !=0 ]
solution

[('q2', 1), ('q3', 1)]